In [1]:
import numpy as np
import time
import random
import scipy.sparse as sps
from scipy.sparse.linalg import svds

In [2]:
fname = "train_triplets.txt"
N = 300000
cutoff = 5
B = 10

In [3]:
def binning(x, bns):
    for b in bns:
        if((x>=b[0]) & (x<=b[1])):
            return b[2]
    return 0

bins = [[np.power(2,i), np.power(2,(i + 1)) - 1, i + 1] for i in range(0, B)]
bins[len(bins) - 1][1] = np.infty

In [4]:
begin = time.clock()

i = 0
data = []
uid = -1
una = ''
with open(fname) as FileObj:
    for line in FileObj:
        if(i >= N):
            break
        a = line.split('\t')
        
        if(a[0] != una):
            uid += 1
            una = a[0]
            
        data.append([uid, int(a[1],36), binning(int(a[2]),bins)])
        #data.append([a[0], a[1], int(a[2])])
        i+=1
data = np.array(data)
data = data[data[:,1].argsort()]
sid = -1
sti = 0
    
for i in range(0, len(data)):
    if(sti != data[i][1]):
        sti = data[i][1]
        sid += 1
    data[i][1] = sid
    
print("Loading the Dataset and took ", time.clock() - begin, " seconds.")

Loading the Dataset and took  1.6870952724820112  seconds.


In [5]:
begin = time.clock()

carryOn = True

while(carryOn):
    
    carryOn = False
    
    uList, uCount = np.unique(data[:,0],return_counts=True)
    sList, sCount = np.unique(data[:,1],return_counts=True)
    cutSongs = np.sort(sList[sCount<=cutoff])
    cutUsers = np.sort(uList[uCount<=cutoff])
    
    delRows = []
    
    if(len(cutSongs) > 0):
        carryOn = True
        data = data[np.argsort(data[:, 1])]
        j = 0
        for i in range(0, len(data)):
            if(data[i][1] > cutSongs[j]):
                j+=1
            if(j >= len(cutSongs)):
                break
            if(data[i][1] == cutSongs[j]):
                delRows.append(i)

    if(len(cutUsers) > 0):
        carryOn = True
        data = data[np.argsort(data[:, 0])]
        j = 0
        for i in range(0, len(data)):
            if(data[i][0] > cutUsers[j]):
                j+=1
            if(j >= len(cutUsers)):
                break
            if(data[i][0] == cutUsers[j]):
                delRows.append(i)
                
    if(len(delRows) > 0):
        delRows = list(set(delRows))
        data = np.delete(data, delRows, axis=0)
        
print("Refining the Dataset and took ", time.clock() - begin, " seconds.")

Refining the Dataset and took  2.8346669990139546  seconds.


In [6]:
begin = time.clock()

data = data[data[:,0].argsort()]
uid = -1
uti = -1

for i in range(0, len(data)):
    if(uti != data[i][0]):
        uti = data[i][0]
        uid += 1
    data[i][0] = uid
    
data = data[data[:,1].argsort()]
sid = -1
sti = -1
    
for i in range(0, len(data)):
    if(sti != data[i][1]):
        sti = data[i][1]
        sid += 1
    data[i][1] = sid
    
M = sps.coo_matrix((data[:,2],(data[:,0],data[:,1])), dtype=np.float).tocsr()

print("Constructing the Matrix took ", time.clock() - begin, " seconds.")

Constructing the Matrix took  0.7160351457257335  seconds.


# Eureka!

use M[i,j] or M[i,:] or M[:,j] format for refinement. use lil matrix for efficiency!

After that, you can finally have fun with GD!

In [13]:
M[0,:] = 0

C:\Program Files (x86)\Anaconda3\lib\site-packages\scipy\sparse\compressed.py:730: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [15]:
for i in range(M.shape[1]):
    print(M[0,i])

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
